In [1]:
import folium
import folium.plugins as plugins
import pandas as pd
import json
import requests
import time
from folium.plugins import HeatMapWithTime
import datetime

In [5]:
#input how long you wish to see
mins = input('How many mins do you want to see? ')
number_of_times = (int(mins)*60)/30

How many mins do you want to see? 2


In [6]:
number_of_times

4.0

In [7]:
cumulative = pd.DataFrame()
startTime = time.time()
for i in range(int(number_of_times)):
    url = ("https://api.data.gov.sg/v1/transport/taxi-availability")
    response = requests.get(url)
    data = response.json()
    df = pd.io.json.json_normalize(data['features'])
    coordinateslist = df['geometry.coordinates'].tolist()
    df1 = pd.DataFrame(coordinateslist)
    result = df1.transpose()
    result.columns = ['coordinates']
    result['Timestamp'] = (df['properties.timestamp'][0])
    cumulative = cumulative.append(result) 
    time.sleep(3)
endTime = time.time()
elapsedTime = endTime - startTime
print("Elapsed Time = %s" % elapsedTime,'seconds')

<ipython-input-7-022fd41b2ecf>:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = pd.io.json.json_normalize(data['features'])


Elapsed Time = 17.690228700637817 seconds


In [8]:
cumulative

,coordinates,Timestamp
0,"[103.62886, 1.29404]",2021-10-26T03:09:01+08:00
1,"[103.63096, 1.32]",2021-10-26T03:09:01+08:00
2,"[103.64281, 1.33441]",2021-10-26T03:09:01+08:00
3,"[103.64558, 1.32721]",2021-10-26T03:09:01+08:00
4,"[103.65688, 1.31398]",2021-10-26T03:09:01+08:00
...,...,...
2211,"[104.00113, 1.38417]",2021-10-26T03:09:31+08:00
2212,"[104.00166, 1.37793]",2021-10-26T03:09:31+08:00
2213,"[104.00227, 1.37597]",2021-10-26T03:09:31+08:00
2214,"[104.00228, 1.37558]",2021-10-26T03:09:31+08:00


In [9]:

#the data has to be formatted into a pandas dataframe from geojson format for easy manipulation
cumulative['coordinates'] = cumulative['coordinates'].astype(str)
Latitude = []
Longitude = []
coordinates = []
for i in cumulative['coordinates']:
    i = i.split(", ")
    lat = i[1][:-1]
    long = i[0][1:]
    Latitude.append(lat)
    Longitude.append(long)
cumulative['Latitude'] = Latitude
cumulative['Longitude'] = Longitude

In [10]:
#converted Timestamp into datetime format and using it as an index
cumulative['Timestamp']= pd.to_datetime(cumulative['Timestamp'])
time_index = []
for i in cumulative['Timestamp'].unique():
    time_index.append(i)
date_strings = [d.strftime('%d/%m/%Y, %H:%M:%S') for d in time_index]

In [11]:
lat_long_list = []
for i in cumulative['Timestamp'].unique():
    temp=[]
    for index, instance in cumulative[cumulative['Timestamp'] == i].iterrows():
        temp.append([instance['Latitude'],instance['Longitude']])
    lat_long_list.append(temp)

In [14]:
date_strings

['26/10/2021, 03:09:01', '26/10/2021, 03:09:31']

In [84]:
lat_long_list[0]

[['1.29404', '103.62886'],
 ['1.32', '103.63096'],
 ['1.33441', '103.64281'],
 ['1.32721', '103.64558'],
 ['1.31398', '103.65688'],
 ['1.31101', '103.65966'],
 ['1.3098', '103.66008'],
 ['1.3104815', '103.660757333333'],
 ['1.31064', '103.66105'],
 ['1.31121', '103.66108'],
 ['1.31158', '103.66183'],
 ['1.32261', '103.66382'],
 ['1.30976', '103.66516'],
 ['1.32032', '103.67509'],
 ['1.3244', '103.67732'],
 ['1.32443', '103.67739'],
 ['1.32436', '103.67745'],
 ['1.32442', '103.67748'],
 ['1.32444', '103.67757'],
 ['1.32436', '103.67761'],
 ['1.32362', '103.67904'],
 ['1.32755', '103.68047'],
 ['1.34045875', '103.685504883333'],
 ['1.34079633333333', '103.688572333333'],
 ['1.34', '103.68881'],
 ['1.34', '103.6907'],
 ['1.34188', '103.69189'],
 ['1.34', '103.69261'],
 ['1.34', '103.69275'],
 ['1.34', '103.69478'],
 ['1.341575', '103.695007'],
 ['1.341358', '103.6951145'],
 ['1.35002', '103.69547'],
 ['1.34558716666667', '103.695683833333'],
 ['1.34666983333333', '103.695708'],
 ['1.35', 

In [12]:
m = folium.Map(location=[1.352083,103.819839],zoom_start = 11, tiles="https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png",attr="Stadia.AlidadeSmoothDark") 

#Plot it on the map
HeatMapWithTime(lat_long_list,radius=5,auto_play=True,position='bottomright',name="cluster",
                index=date_strings,max_opacity=0.7).add_to(m)

# Display the map
m

In [22]:
from folium.plugins import HeatMap

In [27]:
data =[[[    40.7726,    -73.9568,   1900., 1    ],
       [    40.7785,    -73.9556,   3200. ,2   ],
       [    40.7216,    -73.9809,   5800.  ,3  ],
       [    40.7384,    -73.9848,   2900. ,4   ],
       [    40.7678,    -73.9915,   3312.   ,5 ],
       [    40.7659,    -73.9574,   2600. ,6   ],
       [    40.7092,    -74.0137,   4299. ,7   ],
       [    40.7384,    -73.982 ,   5750. ,8   ],
       [    40.7312,    -73.9896,   3595. ,9   ]]]


hmap = folium.Map(location=[40.75, -73.97], tiles='stamentoner',control_scale = True, zoom_start=13)
hmap.add_child(HeatMapWithTime(data, radius = 25))

hmap

In [59]:
import folium
import folium.plugins as plugins
import numpy as np

np.random.seed(3141592)
initial_data = (
    np.random.normal(size=(100, 2)) * np.array([[1, 1]]) +
    np.array([[48, 5]])
)

move_data = np.random.normal(size=(100, 2)) * 0.01

data = [(initial_data + move_data * i).tolist() for i in range(100)]

for time_entry in data:
    for row in time_entry:
        weight = np.random.choice([1, 2, 3, 4])  # default value
        row.append(weight)

In [60]:
data

[[[47.504539036025584, 4.292407029209247, 4],
  [47.69040904564529, 4.252424765823684, 1],
  [46.56622363785289, 3.1698932994737725, 1],
  [48.512345988527414, 3.5575115701151936, 2],
  [47.912356200438545, 4.541688488218782, 4],
  [47.7449114720645, 6.410987139161764, 2],
  [49.27575216702049, 3.583654594233588, 3],
  [47.82845322406549, 3.8219354061097293, 1],
  [48.04493261439567, 6.31593606743332, 4],
  [46.476548975848104, 4.6875193522227105, 4],
  [48.22998111187808, 4.330735827816389, 4],
  [49.09176755111292, 5.097273923224413, 1],
  [48.27319149417738, 5.658018973640203, 1],
  [48.122983644592075, 5.780950867261257, 2],
  [46.44643679370932, 4.755224680829496, 1],
  [49.29166945103235, 6.129019031044898, 3],
  [47.001037365055055, 6.290837811317019, 3],
  [46.8662712198302, 5.128384470003526, 1],
  [48.60872159648292, 4.685247191774352, 4],
  [48.847472424367105, 4.528416690851091, 3],
  [49.82969363846115, 3.7616826932379714, 2],
  [47.41209462975057, 5.286337622398916, 3],
 

In [64]:
len(data)

100

In [56]:
data.shape

AttributeError: 'list' object has no attribute 'shape'

In [78]:
data = [
    
    [[50, 20, 1],
     [50, 21, 10],
     [50, 21.5, 10],
    [51, 22, 3]],
    
    [[49, 24, 2],
     [50, 21, 10],
    [53, 26, 2]],
    
    [[50, 20, 1],
    [51, 22, 3]],
    
    [[49, 24, 2],
    [53, 26, 2]],
    
    [[50, 20, 1],
    [51, 22, 3]],
    
    [[49, 24, 2],
    [53, 26, 2]],
    
]

In [82]:
m = folium.Map([52, 22], tiles='stamentoner', zoom_start=6)

hm = plugins.HeatMapWithTime(data, auto_play=True, min_speed=5.0)

hm.add_to(m)

m

In [50]:
m

In [87]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
import matplotlib as mpl

# parameters
n = 250                     # number of points
lat0 = 40.7                 # coordinates will be generated uniformly with
lon0 = -73.9                # lat0 - eps <= lat < lat0 + eps
eps = 0.1                   # lon0 - eps <= lon < lon0 + eps
v_min, v_max = 0, 100       # min, max values

# generating values
lat = np.random.uniform(lat0 - eps, lat0 + eps, n)
lon = np.random.uniform(lon0 - eps, lon0 + eps, n)
value = np.random.uniform(v_min, v_max, n)
df = pd.DataFrame({'lat': lat, 'lon': lon, 'value': value})

# to demonstrate the effect of weights on the heatmap,
# we'll divide values below the center of the box by K = 5
K = 5
df.loc[df['lat'] < lat0, 'value'] /= K

# plotting the map, both the points themselves and the heatmap
m = folium.Map(location = [lat0, lon0], tiles="OpenStreetMap",
               zoom_start=11, width=400, height=400)
for elt in list(zip(df.lat, df.lon, df.value)):
    folium.Circle(elt[:2], color="white", radius=elt[2]).add_to(m)

# df.values used here is a (250, 3) numpy.ndarray
# with (lat, lon, weight) for each data point
HeatMap(data=df.values, min_opacity=0.1).add_to(m)

m

In [88]:
df

,lat,lon,value
0,40.741405,-73.890874,7.990348
1,40.722987,-73.922341,85.330674
2,40.710454,-73.897659,70.827128
3,40.691915,-73.973888,8.376910
4,40.642532,-73.868822,10.249953
...,...,...,...
245,40.644035,-73.923372,12.536523
246,40.645595,-73.975726,5.214133
247,40.752912,-73.818682,42.205064
248,40.657397,-73.864619,1.644282
